In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.pipeline import Pipeline

In [4]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [15]:
df = pd.read_csv("../data/final_embedded.csv", index_col=0)
df.head()

,LIKES,COMMENTS,target,sin_hour,cos_hour,cos_month,sin_month,cos_weekday,sin_weekday,emb_text_0,...,emb_title_90,emb_title_91,emb_title_92,emb_title_93,emb_title_94,emb_title_95,emb_title_96,emb_title_97,emb_title_98,emb_title_99
0,245,38,0,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.148087,...,0.531877,-0.295999,0.432405,-0.171743,-0.376144,-0.080770,0.780307,-0.393801,0.284733,-0.529976
1,3,12,6,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.095156,...,0.207497,-0.087502,0.449081,0.798268,-0.418835,-0.091712,0.177536,-0.081389,-0.373750,-0.300206
2,73,3,7,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.156003,...,0.600254,-0.218309,0.488272,0.004122,-0.580864,-0.647737,-0.421757,0.404062,-0.599431,-0.755318
3,79,15,7,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.242299,...,0.178026,0.402201,0.071325,-0.381747,-0.243778,-0.708453,0.065975,-0.481236,-0.349474,-0.470402
4,4,10,4,-0.258819,0.965926,1.0,-2.449294e-16,1.0,0.0,0.096658,...,0.045307,0.060338,0.306918,-0.052441,0.083353,-0.437869,0.048654,-0.002049,-0.534763,-0.219268


In [16]:
X, y = df.drop(columns=['target']), df['target']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [18]:
len(X_test)

12853

## Classification

### SVC

In [19]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('svc', SVC())
    ]
)

In [20]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])

In [21]:
pipe.score(X_test, y_test)

0.7829300552400218

### LogisticRegression

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('svc', LogisticRegression())
    ]
)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

/home/memesrized/.virtualenvs/inf_retr/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7541430016338598

### LDA

In [24]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [25]:
pipe = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('svc', LDA())
    ]
)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7369485723177468

## TopicModeling

### LDA

In [22]:
y.unique()

array([0, 6, 7, 4, 3, 1, 2, 5])

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words("russian")

def remove_stop_words(words_list):
    return [x for x in words_list if x not in sw]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/memesrized/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from src.utils.lemmatizers import lemmatizers
import re
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("../data/preprocessed_strict.csv", index_col=0)
df_oh = df[['TEXT', 'target']]

df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: " ".join(lemmatizers['mystem'](x)))
df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: " ".join(remove_stop_words(x.split())))
df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: re.sub('\d+', '0', x))


vectorizer = TfidfVectorizer()

Installing mystem to /home/memesrized/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz
<ipython-input-6-46a90d2ad961>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: " ".join(lemmatizers['mystem'](x)))
<ipython-input-6-46a90d2ad961>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oh['TEXT'] = df_oh['TEXT'].map(lambda x: " ".join(remove_stop_words(x.split())))
<ipython-input-6-46a90d2ad961>:10: SettingWithCopyWarning: 

In [7]:
X_tf_idf = vectorizer.fit_transform(df_oh['TEXT'].tolist())

In [42]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=8, random_state=1)
lda.fit(X_tf_idf)

LatentDirichletAllocation(n_components=8, random_state=1)

In [43]:
vocab = vectorizer.get_feature_names()

n_top_words = 15

topic_words = {}

for topic, comp in enumerate(lda.components_):    
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    print([vocab[x] for x in word_idx],"\n")

['смешнявка', 'петруччо', 'кончит', 'гильберт', 'анный', 'брасс', 'бомжевать', 'сосисочка', 'aaa', 'электромагнит', 'пикапост', 'авторынок', 'френдлент', 'кат', 'педобир'] 

['украина', 'россия', 'страна', 'это', 'который', 'российский', 'украинский', 'сша', 'путин', 'год', 'президент', 'новость', 'война', 'свой', 'власть'] 

['видеокарта', 'i0', 'geforce', 'процессор', 'core', 'intel', '0gb', 'gtx', 'amd', 'nvidia', 'проц', '0m', 'комма', 'asus', 'radeon'] 

['the', 'you', 'and', 'to', 'is', 'of', 'in', 'that', 'not', 'with', 'your', 'it', 'this', 'was', 'we'] 

['вятский', 'квас', 'vga', 'михрютка', 'манту', 'кл', 'zzz', 'вилма', 'злопамятный', 'vaio', 'пиздокрывать', 'сакура', 'фазан', 'клаустрофоб', 'king'] 

['это', 'весь', 'мочь', 'свой', 'человек', 'девушка', 'который', 'год', 'знать', 'очень', 'друг', 'ребенок', 'хотеть', 'помогать', 'сказать'] 

['йоттар', 'coub', 'фш', 'utm', 'ремейк', 'power', 'новичка', 'табас', 'пидр', 'view', 'уот', 'непрокатить', 'усраина', 'amp', 'share

In [53]:
vocab = vectorizer.get_feature_names()

n_top_words = 10

topic_words = {}

for topic, comp in enumerate(lda.components_):    
    word_idx = np.argsort(comp)[::-1][:n_top_words]
    print([vocab[x] for x in word_idx],"\n")

['минус', 'помогать', 'коммент', 'пожалуйста', 'внутри', 'пикабушник', 'помощь', 'ссылка', 'комментарий', 'пост'] 

['украина', 'россия', 'страна', 'российский', 'украинский', 'сша', 'это', 'путин', 'который', 'президент'] 

['рэнд', 'шанни', 'стефан', 'некромантка', 'риш', 'рутта', 'deleted', 'point', 'крей', 'скьолвы'] 

['мопс', 'анкетирование', 'шамбамбукля', 'мазукт', 'скумбрия', 'антифриз', 'укулеля', 'макрос', 'павлодар', 'лавсан'] 

['error', 'маскоу', 'survey', 'survio', 'губерман', 'вятский', 'vaio', 'трансформер', 'range', 'rover'] 

['это', 'весь', 'свой', 'человек', 'мочь', 'который', 'девушка', 'год', 'знать', 'ребенок'] 

['винд', 'видеокарта', 'windows', 'драйвер', 'the', 'i0', 'гб', 'you', 'asus', 'ноутбук'] 

['радмир', 'coub', 'del', 'леголас', 'табас', 'боян', 'кончит', 'мэд', 'ляксей', 'ширшина'] 



## Clustering

### Kmeans-fasttext

In [26]:
from collections import Counter
from sklearn.cluster import KMeans
from sklearn import metrics

In [27]:
kmeans = KMeans(n_clusters=8).fit(X)

In [28]:
kmeans.predict(X)

array([3, 1, 1, ..., 1, 3, 3], dtype=int32)

In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder().fit(y)

In [30]:
np.unique(kmeans.predict(X))

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32)

In [31]:
Counter(kmeans.predict(X)).most_common()

[(1, 44800),
 (3, 2920),
 (5, 1453),
 (0, 1008),
 (6, 638),
 (4, 404),
 (2, 158),
 (7, 29)]

In [32]:
Counter(y).most_common()

[(4, 13169),
 (3, 9441),
 (7, 9273),
 (1, 4864),
 (5, 4763),
 (6, 4357),
 (0, 3262),
 (2, 2281)]

In [33]:
le.transform(y)

array([0, 6, 7, ..., 7, 1, 7])

In [34]:
metrics.adjusted_rand_score(kmeans.predict(X), le.transform(y))

0.013200719914041453

### Kmeans-tfifd

In [35]:
km_tfidf=KMeans(n_clusters=8).fit(X_tf_idf)

In [36]:
X_tf_idf.shape

(51416, 151580)

In [37]:
metrics.adjusted_rand_score(km_tfidf.predict(X_tf_idf), le.transform(df_oh['target']))

0.16233893715582967

### OPTICS

In [ ]:
from sklearn.cluster import OPTICS

opt = OPTICS().fit(X)